![](header.jpg)

# eCompass

![](frames.png)

A magnetometer measures the Earth (or any) magnetic field. Unfortunately, the Earth's magnetic field is very weak and can be obscured by electromagnetic interferance from:

- motors
- wifi
- power lines
- metal located in the frame of a building you are in side (think [faraday cage](https://en.wikipedia.org/wiki/Faraday_cage))

The magnetic field measured in both the world and body frame are:

$$
b^W = B \begin{bmatrix} \cos(I) & 0 & \sin(I) \end{bmatrix}^T \\
b^B = R^B_W b^W = \begin{bmatrix} b_x & b_y & b_z \end{bmatrix}^T
$$

where $b^W$ is the magnetic field in the world frame, $b^B$ is the field in the body frame, $B$ is the magnetude of the magnetic field vector $b$, and $I$ is the inclination of the magnetic field. Notice how the world frame field is only defined in the x and z axis, there is no field in they y. However, your sensor measures in the body frame and will sense on all three axes depending on orientation.

> Note: if you know your latitude and longitude, you can look up $B$ and $I$

Now, given that $R^W_B = R_z R_y R_x$ and you can determine the roll and pitch of your magnetometer from an accelerometer (they are often sold together in one package), you can determine your heading. If you multiply this together to, your heading is:

$$
\psi = \text{atan2}(\cos(\theta)(b_z\sin(\phi)-b_y\cos(\phi)), b_x+B\sin(I)\sin(\theta))
$$

The [rotation matrix](https://www.mathworks.com/help/fusion/ref/ecompass.html#mw_ecf9e057-074a-4fe7-b63e-0d50d95946a0) is also given by:

$$
R = 
\begin{bmatrix}
((a \times m) \times a && a \times m && a
\end{bmatrix}
$$

then normalized across the columns.

## Calibration

This notebook doesn't cover calibration of the magnetometer for hard or soft iron. The assumption here is, the magnetometer is already calibrated.

## References

- Mathworks: [ecompass](https://www.mathworks.com/help/fusion/ref/ecompass.html#mw_55a75a75-2bfc-4890-9d2d-42e3258c71e5)
- Github: [Open source sensor fusion](https://github.com/memsindustrygroup/Open-Source-Sensor-Fusion)
- Robot Academy: [How magnetometers work](https://robotacademy.net.au/lesson/how-magnetometers-work/)
- Robot Academy: [Using magnetometers](https://robotacademy.net.au/lesson/using-magnetometers/)

In [1]:
# import a bunch of libraries
import numpy as np 
from numpy import cos, sin, pi, sqrt
from numpy import arcsin as asin
from numpy import arccos as acos
from numpy import arctan2 as atan2
from numpy.linalg import norm
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

from scipy import linalg
import sys

import the_collector # read data
print(f"the-collector: {the_collector.__version__}")

from the_collector import BagIt
from the_collector import Pickle, Json

from squaternion import Quaternion

import pandas as pd

%matplotlib inline
from matplotlib import pyplot as plt

# from math import sqrt, atan2, asin, pi
from math import radians as deg2rad
from math import degrees as rad2deg

from slurm import storage

from datetime import datetime
import os

the-collector: 0.8.7


In [2]:
# let's load in some data and have a look at what we have
def bag_info(bag):
    print('Bag keys:')
    print('-'*50)
    for k in bag.keys():
        print(f'  {k:>10}: {len(bag[k]):<7}')

bag = BagIt(Pickle)
fname = "~/github/data-ins-1/2020-5-3-imu/still-z-up.pickle.bag"
fname = os.path.expanduser(fname)

data = bag.read(fname)
bag_info(data)

>> Reading[pickle]: /Users/kevin/github/data-ins-1/2020-5-3-imu/still-z-up.pickle.bag
Bag keys:
--------------------------------------------------
       accel: 2001   
         mag: 2001   
        gyro: 2001   


In [14]:
def normalize(x, y, z):
    """Return a unit vector"""
    norm = sqrt(x * x + y * y + z * z)

    # already a unit vector
    if norm == 1.0:
        return (x, y, z)

    inorm = 1.0/norm
    if inorm > 1e-6:
        x *= inorm
        y *= inorm
        z *= inorm
    else:
        raise ZeroDivisionError(f'norm({x:.4f}, {y:.4f}, {z:.4f},) = {inorm:.6f}')
    return (x, y, z,)

def ecompass(accel, mag):
    try:
        mx, my, mz = mag
        ax, ay, az = normalize(*accel)

        pitch = asin(-ax)

        if abs(pitch) >= pi/2:
            roll = 0.0
        else:
#             roll = asin(ay/cos(pitch))
            roll = atan2(ay,az)

        # mx, my, mz = mag
        x = mx*cos(pitch)+mz*sin(pitch)
        y = mx*sin(roll)*sin(pitch)+my*cos(roll)-mz*sin(roll)*cos(pitch)
        heading = atan2(y, x)

        # wrap heading between 0 and 360 degrees
        heading = heading % (2*pi)

        return Quaternion.from_euler(roll, pitch, heading)

    except ZeroDivisionError:
        raise

def ecompass_rot(a,m):
    # see Matlab reference
    n = np.cross(a,m)
    m = np.cross(n,a)
    o = a
    
    oo = norm(o)
    if oo < 1e-6:
        raise Exception("eCompass: accel == 0.0")

    m = m/norm(m)
    n = n/norm(n)
    o = o/oo

    r = np.array([m,n,o]).T
    return r

In [15]:
# https://www.mathworks.com/help/fusion/ref/ecompass.html
# m = [19.535 -5.109 47.930];
# a = [0 0 9.8];
# q = ecompass(a,m);
# quaterionEulerAngles = eulerd(q,'ZYX','frame')
# 14.6563         0         0
m = np.array([19.535, -5.109, 47.930])
a = np.array([0, 0, 9.8])
q = ecompass(a,m)
print(q)
print(q.to_euler(degrees=True))
print(np.array(q.to_rot()))

Quaternion(w=-0.5868195702546446, x=-0.5868195702546445, y=0.3945158957078337, z=0.39451589570783374)
(89.99999999999999, 0.0, -67.82549073634752)
[[ 0.377  0.    -0.926]
 [-0.926  0.    -0.377]
 [ 0.     1.     0.   ]]


In [9]:
# https://www.mathworks.com/help/fusion/ref/ecompass.html#mw_ecf9e057-074a-4fe7-b63e-0d50d95946a0
m = np.cross(np.cross(a,m),a)
n = np.cross(a,m)
o = a

m = m/np.linalg.norm(m)
n = n/np.linalg.norm(n)
o = o/np.linalg.norm(o)

rr = np.array([m,n,o]).T
print(rr)

[[ 0.967  0.253  0.   ]
 [-0.253  0.967  0.   ]
 [ 0.    -0.     1.   ]]


In [10]:
a = np.array([0,0,1])
r = ecompass_rot(a,m)
print(r)
print(np.linalg.det(r))

theta = -asin(r[0,2])
ct = 1/cos(theta)
psi = atan2(r[1,2]*ct,r[2,2]*ct)
rho = atan2(r[0,1]*ct,r[0,0]*ct)
print(f">> {rho*180/pi} {theta*180/pi} {psi*180/pi}")

[[ 0.967  0.253  0.   ]
 [-0.253  0.967  0.   ]
 [ 0.    -0.     1.   ]]
1.0
>> 14.656328776110113 -0.0 0.0
